
### Bulk data downloads from NSIDC
* https://nsidc.org/support/faq/what-options-are-available-bulk-downloading-data-https-earthdata-login-enabled#mac
 
Given the instructions for downloading data using `wget` in the link above, we can utilize python to make the process even simplier. Summarizing the steps given in the link above:  
    
1. Configure your username and password for authentication using a .netrc file
```
        >cd ~
        >touch .netrc
        >echo "machine urs.earthdata.nasa.gov login <uid> password <password>" >> .netrc
        >chmod 0600 .netrc
```
2. Create a cookie file.
```
        >touch .urs_cookies
```
3. Use WGET to download the data
```
        >wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --keep-session-cookies --no-check-certificate --auth-no-challenge=on -r --reject "index.html*" -np -e robots=off <insert complete HTTPS data URL>
```

## 1.1 Access SMAP data via Wget

### Make sure you have a recent version of wget.

* We are going to do this in python, but you can do this in other languages also.

* Click [here](https://www.gnu.org/software/wget/) to download WGET and a description about the software

* SMAP data are available through the following url:
    * https://n5eil02u.ecs.nsidc.org/opendap/SMAP/

* For these examples, we will use **SPL3SMP version 004**, but the same methdology to access and subset SMAP data will work for any of the products availabe there

import packages

In [ ]:
import os
import xarray as xr
import h5py
import numpy as np

### Downloading March 2017 data 

1. Create and navigate to a Data directory that will contain the files we download.

In [ ]:
data_out = './Data/L3_SM_P/'

def check_directory(destination_directory):
    if not os.path.exists(destination_directory):
        print('creating directory: ' + destination_directory)
        os.makedirs(destination_directory)
    else:
        print('directory exists: ' + destination_directory)
    return None

check_directory(data_out)
os.chdir(data_out)
cwd = os.getcwd()
print(cwd)

#### Now we will create a function to return the url and filename of the data we want to download.

For SMAP we will do this by year, month, and day. The rest of the filename is predictable.

In [ ]:
def SMAP_L3_P_36km_Path(year, month, day):
    fpath_start = 'https://n5eil01u.ecs.nsidc.org/SMAP/'
    host = 'https://n5eil01u.ecs.nsidc.org/'
    version = '.004'
    url_path = '{host}/SMAP/SPL3SMP{version}/{year}.{month:02}.{day:02}/'.format(host=host,
                                                                                 version=version,
                                                                                 year=year,
                                                                                 month=month,
                                                                                 day=day)
    filename = 'SMAP_L3_SM_P_{year}{month:02}{day:02}_R14010_001.h5'.format(year=year, 
                                                                            month=month, 
                                                                            day=day)

    smap_data_path = url_path + filename
    
    return smap_data_path, filename


We take the example from the NSIDC wget command above and append our file name to this.

In [ ]:
WGET_COMMAND_START = 'wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --keep-session-cookies --no-check-certificate --auth-no-challenge=on -r --reject "index.html*" -np -e robots=off '

## Download the data

In [ ]:
import calendar

# Download data for March 2017
year = 2017
month = 3
_, days_in_month = calendar.monthrange(2017, 3)  # 31 days in March 2017

for day in range(1, days_in_month + 1):
    print('opening SMAP data for: '+str(year)+'-'+str(month).zfill(2)+'-'+str(day).zfill(2))
    
    full_path,fileName = SMAP_L3_P_36km_Path(year, month, day)
    WGET_COMMAND = WGET_COMMAND_START+'-O '+fileName+' '+full_path
    print(WGET_COMMAND)
    os.system(WGET_COMMAND)
    
    print(fileName + ' downloaded')
    print('*** SM data saved to: '+cwd + fileName+' *** ')

### Now you have the tools to download in bulk!

**FYI these scripts can take a long time, especially if you are downloading larger files.**
* SMAP 36km data are ~ 30MB
* SMAP 9km data are ~ 300MB
* SMAP 3km data are ~ 2 GB

** This is why OPENDAP makes more sense in some cases **